In [2]:
from PIL import Image
from io import BytesIO
import io
from kafka import KafkaConsumer
from hdfs import InsecureClient
import cv2
import imutils
import pytesseract

In [3]:
consumer = KafkaConsumer("test",bootstrap_servers=['localhost:9092'],api_version=(0,10,1))
client = InsecureClient('http://localhost:9870')
client.makedirs('/path/to/hdfs/directory')
pytesseract.pytesseract.tesseract_cmd = 'C:\Program Files\Tesseract-OCR\\tesseract'

In [7]:
from io import BytesIO
from PIL import Image
import pytesseract
from hdfs import InsecureClient
from kafka import KafkaConsumer
import json
import base64
from datetime import datetime
# Connect to HDFS client
client = InsecureClient('http://localhost:9870')
# Connect to Kafka consumer
consumer = KafkaConsumer(
    'test',
    bootstrap_servers=['localhost:9092'],
    value_deserializer=lambda m: json.loads(m.decode('utf-8')))

i = 1
# Loop through Kafka messages
for message in consumer:
    # Extract filename and image bytes from the message
    filename = message.value['filename']
    image_bytes = message.value['image'].encode('utf-8')
    image = Image.open(BytesIO(base64.b64decode(image_bytes)))

    text = filename[:-4]

    file_list = client.list('/path/to/hdfs/')
    value = True
    for file in file_list:
        if file != 'directory':
            with client.read(f'/path/to/hdfs/{file}', encoding='utf-8') as reader:
                contents = reader.read()
                json_dict = json.loads(contents)
                if json_dict['number_plate_value'] == text:
                    date_string = json_dict['datetime']
                    dt1 = datetime.strptime(date_string,"%Y-%m-%d %H:%M:%S.%f")
                    dt2 = datetime.now()
                    diff = dt2 - dt1
                    price = round(((diff.total_seconds() / 3600)*20),2)
                    print(f"Pay {price}$")
                    client.delete(f'/path/to/hdfs/{file}')
                    value = False
    if value:
        number_plate_value = text
        data = {
            "datetime": str(datetime.now()),
            "number_plate_value": number_plate_value,
            "number_plate_image": str((image_bytes), "utf-8")
            }
        with client.write(f'/path/to/hdfs/output{i}.json', encoding='utf-8') as writer:
            json.dump(data, writer)
        with client.read(f'/path/to/hdfs/output{i}.json', encoding='utf-8') as reader:
            contents = reader.read()
        print(contents)
        data_dict = json.loads(contents)
        datetime_str = data_dict['datetime']
        number_plate_value = data_dict['number_plate_value']
        number_plate_image_str = data_dict['number_plate_image']
        number_plate_image_bytes = base64.b64decode(number_plate_image_str)
        number_plate_image = Image.open(BytesIO(number_plate_image_bytes))
        #number_plate_image.show()
        print(datetime_str)
        print(number_plate_value)
        i += 1          

{"datetime": "2023-05-03 15:22:06.603127", "number_plate_value": "", "number_plate_image": "..."}
2023-05-03 15:22:06.603127

{"datetime": "2023-05-03 15:22:16.565456", "number_plate_value": "21 BH 2345 AA", "number_plate_image": "..."}
2023-05-03 15:22:16.565456
21 BH 2345 AA
{"datetime": "2023-05-03 15:22:16.930207", "number_plate_value": "5253", "number_plate_image": "..."}
2023-05-03 15:22:16.930207
5253
{"datetime": "2023-05-03 15:22:17.296002", "number_plate_value": "9993", "number_plate_image": "/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQECAgICAgQDAgICAgUEBAMEBgUGBgYFBgYGBwkIBgcJBwYGCAsICQoKCgoKBggLDAsKDAkKCgr/2wBDAQICAgICAgUDAwUKBwYHCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgoKCgr/wAARCABwAOkDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW

KeyboardInterrupt: 

In [9]:
x = 1.213123213
print(x)

1.213123213


In [11]:
print(round(x,2))

1.21
